# Deep Learning Toolkit for Splunk - Notebook for STL - Seasonality and Trend Decomposition

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import STL
import statsmodels as sm
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("statsmodels version: " + sm.__version__)

numpy version: 1.19.2
pandas version: 1.1.3
statsmodels version: 0.12.0


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| inputlookup cyclical_business_process.csv<br>
| fit MLTKContainer mode=stage algo=seasonality_and_trend_decomposition _time logons

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [63]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [74]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("default")
print(df.describe())
print(param)

             logons
count  11042.000000
mean      55.689006
std       44.706658
min        0.000000
25%       18.000000
50%       42.000000
75%       86.000000
max      286.200000
{'options': {'params': {'mode': 'stage', 'algo': 'seasonality_and_trend_decomposition'}, 'args': ['*'], 'feature_variables': ['*'], 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '1200', 'max_inputs': '500000', 'max_memory_usage_mb': '6000', 'max_model_size_mb': '15', 'max_score_time': '600', 'streaming_apply': '0', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['_time', 'logons']}


## Stage 2 - create and initialize a model

In [13]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    return model

In [17]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))
model=init(df,param)

{}


## Stage 3 - fit the model

In [18]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    return "info"

In [19]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

info


## Stage 4 - apply the model

In [149]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    data=df
    data['_time']=pd.to_datetime(data['_time'])
    data = data.set_index('_time') # Set the index to datetime object.
    data=data.asfreq('H')
    
    res=STL(data).fit()
    results=pd.DataFrame({"seasonality": res.seasonal, "trend": res.trend, "residual": res.resid})
    results.reset_index(level=0, inplace=True)
    return results

In [150]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
apply(model,df,param)

,seasonality,trend,residual
_time,,,
2018-08-04 00:00:00-07:00,-18.539007,21.048721,-2.509714
2018-08-04 01:00:00-07:00,-16.283760,21.414126,-2.130367
2018-08-04 02:00:00-07:00,-14.912778,21.772331,-2.859553
2018-08-04 03:00:00-07:00,-15.487972,22.122890,-2.634919
2018-08-04 04:00:00-07:00,-17.417354,22.465436,-0.048082
...,...,...,...
2018-09-11 04:00:00-07:00,-40.159651,64.758349,-9.598698
2018-09-11 05:00:00-07:00,-37.880478,64.413210,-8.532731
2018-09-11 06:00:00-07:00,-24.266634,64.036520,-1.769886


## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code